In [16]:
import pandas as pd
import joblib  # For loading .pkl files
import requests  # For making API calls

# Load the trained Random Forest model, scaler, and label encoder
rf_model = joblib.load('gb_model.pkl')  # Replace with the correct path if needed
scaler = joblib.load('scaler.pkl')     # Replace with the correct path if needed
label_encoder = joblib.load('label_encoder.pkl')  # Replace with the correct path if needed

# Blynk API URLs for real-time sensor values and result upload
tds = "https://blynk.cloud/external/api/get?token=FbQw1KDQ3w-5698oJsygi9QC13xrWxng&V0"
turbidity = "https://blynk.cloud/external/api/get?token=FbQw1KDQ3w-5698oJsygi9QC13xrWxng&V1"
pH = "https://blynk.cloud/external/api/get?token=FbQw1KDQ3w-5698oJsygi9QC13xrWxng&V2"
mq4 = "https://blynk.cloud/external/api/get?token=FbQw1KDQ3w-5698oJsygi9QC13xrWxng&V3"
temperature = "https://blynk.cloud/external/api/get?token=FbQw1KDQ3w-5698oJsygi9QC13xrWxng&V4"
humidity = "https://blynk.cloud/external/api/get?token=FbQw1KDQ3w-5698oJsygi9QC13xrWxng&V5"
rain = "https://blynk.cloud/external/api/get?token=FbQw1KDQ3w-5698oJsygi9QC13xrWxng&V6"
RESULT_UPLOAD_URL = "https://blynk.cloud/external/api/update?token=FbQw1KDQ3w-5698oJsygi9QC13xrWxng&V8={}"
RESULT_UPLOAD_URL1 = "https://blynk.cloud/external/api/update?token=FbQw1KDQ3w-5698oJsygi9QC13xrWxng&V11={}"

def get_sensor_value(url):
    response = requests.get(url)
    if response.status_code == 200:
        return float(response.text.strip())
    else:
        raise Exception(f"Failed to fetch data from {url}. HTTP Status Code: {response.status_code}")

def upload_result_to_blynk(result):
    response = requests.get(RESULT_UPLOAD_URL.format(result))
    if response.status_code == 200:
        print(f"Result '{result}' successfully uploaded to Blynk.")
    else:
        raise Exception(f"Failed to upload result to Blynk. HTTP Status Code: {response.status_code}")


def upload_result_to_blynk1(result):
    response = requests.get(RESULT_UPLOAD_URL1.format(result))
    if response.status_code == 200:
        print(f"Result '{result}' successfully uploaded to Blynk.")
    else:
        raise Exception(f"Failed to upload result to Blynk. HTTP Status Code: {response.status_code}")

def disaster(tds, turbidity, pH, mq4, temperature, humidity, rain):

    # Create a DataFrame from the input values
    new_data = pd.DataFrame({
        'tds': [tds],
        'turbidity': [turbidity],
        'pH': [pH],
        'mq4': [mq4],
        'tempre': [temperature],
        'humidity': [humidity],
        'rain': [rain]
    })
    
    # Scale the new data using the loaded scaler
    new_data_scaled = scaler.transform(new_data)
    
    # Make predictions using the loaded Random Forest model
    dt_prediction = rf_model.predict(new_data_scaled)
    
    # Convert numerical prediction back to the original label
    result = label_encoder.inverse_transform(dt_prediction)
    
    return result[0]

# Main function to fetch real-time sensor values, make a prediction, and upload the result
if __name__ == "__main__":
    try:
        print("Fetching real-time sensor values...")
        
        # Fetch real-time sensor values
        tds = get_sensor_value(tds)
        turbidity = get_sensor_value(turbidity)
        pH = get_sensor_value(pH)
        mq4 = get_sensor_value(mq4)
        temperature = get_sensor_value(temperature)
        humidity = get_sensor_value(humidity)
        rain = get_sensor_value(rain)
        
        print(f"tds: {tds}")
        print(f"turbidity: {turbidity}")
        print(f"pH: {pH}")
        print(f"mq4: {mq4}")
        print(f"temperature: {temperature}")
        print(f"humidity: {humidity}")
        print(f"rain: {rain}")
        
        # Predict the disaster type
        predicted_result = disaster(tds, turbidity, pH, mq4, temperature, humidity, rain)
        
        # Display the prediction
        print("\nPredicted Result:", predicted_result)

        if predicted_result=="Low":
            upload_result_to_blynk1(1)
        else:
            upload_result_to_blynk1(0)

        # Upload the prediction result to Blynk
        upload_result_to_blynk(predicted_result)
        
    except Exception as e:
        print("\nError:", str(e))


Fetching real-time sensor values...
tds: 100.0
turbidity: 0.0
pH: 8.0
mq4: 49.0
temperature: 32.0
humidity: 71.0
rain: 0.0

Error: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- tempre
Feature names seen at fit time, yet now missing:
- temperature



In [18]:
print("Successfully fetched the data")

Successfully fetched the data
